In [1]:
import requests
from helper import *
from selenium import webdriver

from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.wait import WebDriverWait
import selenium.webdriver.support.expected_conditions as EC
from selenium.webdriver.common.by import By

import time

In [2]:
#info

#načeloma se sprimenja samo te podatke

username = "ime.priimek@portal.skavt.net" #TODO
password = "geslo" #TODO

url_home = "domača stran"#"https://preddvor1.skavt.net/" TODO
url_login = url_home+"login"

driver_path = "lokacija chroma"#C:/Program Files/Google/Chrome/Application/chrome.exe" TODO
media_loc = r'lokacija za slike in druge medije'#r"C:/Users/Karel/Documents/Skavti/spletna stran arhiv/media/" TODO

In [3]:
#login
#https://preddvor1.skavt.net/login

# Load browser and opening page
option = webdriver.ChromeOptions()
option.add_argument('--incognito')

browser = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=option)
browser.set_window_size(1500, 910)

browser.get(url_home)

#login
login_elem = None
a_elems = browser.find_elements('tag name', 'a')
for a in a_elems:
    if a.text == 'Prijava':
        login_elem = a
        break
if login_elem is None:
    print('Failed to find Login Button')
else:
    login_elem.click()

username_elem = None
password_elem = None
WebDriverWait(browser, 20).until(EC.presence_of_all_elements_located((By.TAG_NAME, 'input')))
input_elems = browser.find_elements('tag name', 'input')

# Iterate through input elements
for i in input_elems:
    print(i.get_attribute('placeholder'))
    if i.get_attribute('placeholder') == 'Uporabniško ime':
        username_elem = i
    if i.get_attribute('placeholder') == 'Geslo':
        password_elem = i

# Check that valid elements are returned
if username_elem is None:
    print('Failed to find username field')
if password_elem is None:
    print('Failed to find password field')

# Populate fields
if username_elem is not None and password_elem is not None:
    username_elem.send_keys(username)
    password_elem.send_keys(password)

#login button
WebDriverWait(browser, 20).until(EC.presence_of_all_elements_located((By.TAG_NAME, 'button')))
login_button = browser.find_element('tag name', 'button')
login_button.click()

#administrative button
WebDriverWait(browser, 20).until(EC.presence_of_all_elements_located((By.TAG_NAME, 'a')))

time.sleep(2)

browser.get(url_home+'admin')

Uporabniško ime
Geslo


In [4]:
def find_a(browser, href='', text=''):
    if href!= '':
        a = browser.find_element(By.XPATH, "//a[@href='"+href+"']")
        return a
    if text != '':
        a = browser.find_element(By.XPATH, "//a[text()='"+text+"']")
        return a
    return False

def find_all_a_by_class(browser, clss=''):
    return browser.find_elements(By.CLASS_NAME, 'post_title')

def collect_post(browser):
    browser.find_element(By.ID, 'categories').click()
    tags = browser.find_elements(By.XPATH, "//li[@class='selected']/label") #or selected hover
    tags = tags + browser.find_elements(By.XPATH, "//li[@class='selected hover']/label")
    tags = [t.text for t in tags]
    
    #browser.find_element(By.CLASS, 'month').click()
    month = browser.find_elements(By.XPATH, "//option[@selected='selected']")
    month = [m.get_attribute('value') for m in month if (len(m.text)==3)][0]
    day = browser.find_element(By.ID, 'publish_date').get_attribute('value')
    year = browser.find_element(By.ID, 'publish_year').get_attribute('value')

    author =  browser.find_element(By.ID, 'author').get_attribute('value')
    title = browser.find_element(By.ID, 'title').get_attribute('value')
    abstract = browser.find_element(By.ID, 'excerpt').text
    
    browser.switch_to.frame(browser.find_element(By.ID, 'content_ifr'))
    body = browser.find_element(By.ID, 'tinymce').text
    images = browser.find_elements(By.TAG_NAME, 'img')
    images = [i.get_attribute('src').split('/')[-1].split('?')[0] for i in images]
    browser.switch_to.default_content()

    #creating text
    text = f"Naslov: {title}\n\nZnačke: {tags}\n\nAvtor: {author}\n\nDatum objave: {day}. {month}. {year}\n\nPovzetek: {abstract}\n\nVsebina: {body}\n\nSlike:\n{images}\n---\n "
    
    return text

def collect_page(browser):
    month = browser.find_elements(By.XPATH, "//option[@selected='selected']")
    month = [m.get_attribute('value') for m in month if (len(m.text)==3)][0]
    day = browser.find_element(By.ID, 'publish_date').get_attribute('value')
    year = browser.find_element(By.ID, 'publish_year').get_attribute('value')

    title = browser.find_element(By.ID, 'title').get_attribute('value')
    
    browser.switch_to.frame(browser.find_element(By.ID, 'content_ifr'))
    body = browser.find_element(By.ID, 'tinymce').text
    images = browser.find_elements(By.TAG_NAME, 'img')
    images = [i.get_attribute('src').split('/')[-1].split('?')[0] for i in images]
    browser.switch_to.default_content()

    #creating text
    text = f"Naslov: {title}\n\nDatum objave: {day}. {month}. {year}\n\n Vsebina: {body}\n\n Slike:\n{images}\n---\n "
    
    return text

def collect_media(browser):
    title = browser.find_element(By.NAME, 'upload_title[]').get_attribute('value')

    link = browser.find_element(By.XPATH, '//div[@id="center"]/form/div/a').get_attribute('href')
    if title == '':
        title = link.split('/')[-1]
    return (title, link)

In [5]:
#objave

file = open('objave.txt', 'w+')

#click urejaj
edit = find_a(browser, href='admin/news/browse')
edit.click()

#iterate pages
ith = 1
while True:
    #iterate articles on a page
    all_a = find_all_a_by_class(browser, 'post_title')
    for i in range(len(all_a)):

        time.sleep(1)

        all_a = find_all_a_by_class(browser, 'post_title')
        all_a[i].click()

        #collect data
        file.write(collect_post(browser))

        time.sleep(1)

        browser.back()

        print("Downloaded: " + str(ith))
        ith += 1

    try:
        time.sleep(2)
        repeat = browser.find_element(By.CLASS_NAME, 'next')
        repeat.click()
    except:
        break

file.close()
    

Downloaded: 1
Downloaded: 2
Downloaded: 3
Downloaded: 4
Downloaded: 5
Downloaded: 6
Downloaded: 7
Downloaded: 8
Downloaded: 9
Downloaded: 10
Downloaded: 11
Downloaded: 12
Downloaded: 13
Downloaded: 14
Downloaded: 15
Downloaded: 16
Downloaded: 17
Downloaded: 18
Downloaded: 19
Downloaded: 20
Downloaded: 21
Downloaded: 22
Downloaded: 23
Downloaded: 24
Downloaded: 25
Downloaded: 26
Downloaded: 27
Downloaded: 28
Downloaded: 29


In [6]:
#strani

file = open('strani.txt', 'w+')

#click urejaj
edit = find_a(browser, href='admin/page/browse')
edit.click()

#iterate pages
ith = 1
while True:
    #iterate articles on a page
    all_a = find_all_a_by_class(browser, 'post')
    for i in range(len(all_a)):

        time.sleep(1)

        all_a = find_all_a_by_class(browser, 'post')
        all_a[i].click()

        #collect data
        file.write(collect_page(browser))

        time.sleep(1)

        browser.back()

        print("Downloaded: " + str(ith))
        ith += 1

    try:
        time.sleep(2)
        repeat = browser.find_element(By.CLASS_NAME, 'next')
        repeat.click()
    except:
        break

file.close()
    

Downloaded: 1
Downloaded: 2
Downloaded: 3
Downloaded: 4
Downloaded: 5
Downloaded: 6
Downloaded: 7
Downloaded: 8
Downloaded: 9
Downloaded: 10


In [7]:
#mediji

#click urejaj
edit = find_a(browser, href='admin/media/browse')
edit.click()

#iterate pages
ith = 1
while True:
    #iterate articles on a page
    all_a = find_all_a_by_class(browser, 'post_title')
    for i in range(len(all_a)):

        time.sleep(1)

        all_a = find_all_a_by_class(browser, 'post_title')
        all_a[i].click()

        #collect data
        (name, url) = collect_media(browser)
        data = requests.get(url).content

        loc = media_loc+name

        with open(loc, 'wb') as f:
            f.write(data)
        
        

        time.sleep(1)

        browser.back()

        print("Downloaded: " + str(ith))
        ith += 1

    try:
        time.sleep(2)
        repeat = browser.find_element(By.CLASS_NAME, 'next')
        repeat.click()
    except:
        break
    

Downloaded: 1
Downloaded: 2
Downloaded: 3
Downloaded: 4
Downloaded: 5
Downloaded: 6
Downloaded: 7
Downloaded: 8
Downloaded: 9
Downloaded: 10
Downloaded: 11
Downloaded: 12
Downloaded: 13
Downloaded: 14
Downloaded: 15
Downloaded: 16
Downloaded: 17
Downloaded: 18
Downloaded: 19
Downloaded: 20
Downloaded: 21
Downloaded: 22
Downloaded: 23
Downloaded: 24
Downloaded: 25


KeyboardInterrupt: 